# <span style="color:blue">Delaware Valley Region School Atlas</span>

This atlas visualizes schools in the Delaware Valley region. The map plots schools based on their type, focusing on the following four categories:

- **District Public** (Blue)
- **Charter Public** (Green)
- **Secular Private** (Purple)
- **Religious Private** (Red)

The color-coded markers represent each school type, providing a clear geographical overview.


In [1]:
# Basic Imports
import os
import zipfile
import time

# Data Management Imports
import pandas as pd
import openpyxl

# Visualizations
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim

In [5]:
### CREATING THE GEODATAFRAME FOR BOUNDARIES OF DEL.VAL. REGION

# Making path to the folder where shapefiles are located
shapefile_folder = './shapefiles' 

# Find the shapefile (.shp) in the folder
shapefile_path = next((os.path.join(shapefile_folder, f) for f in os.listdir(shapefile_folder) if f.endswith('.shp')), None)

if shapefile_path:
    # Read the shapefile using GeoPandas
    gdf = gpd.read_file(shapefile_path)

    # Check the CRS (Coordinate Reference System) and reproject if necessary
    if gdf.crs != 'EPSG:4326':
        gdf = gdf.to_crs(epsg=4326)  # Reproject to EPSG:4326


    # Filter the GeoDataFrame for counties in the Delaware Valley region
    delaware_valley_counties = [
        "Philadelphia", "Bucks", "Chester", "Delaware", "Montgomery",  # Pennsylvania counties
        "Burlington", "Camden", "Gloucester", "Mercer",  # New Jersey counties
        "New Castle",  "Kent" # Delaware county
    ]
    
    delaware_valley_gdf = gdf[gdf['co_name'].isin(delaware_valley_counties)]
    delaware_valley_gdf = delaware_valley_gdf[delaware_valley_gdf.is_valid]

    # print("Counties in the Delaware Valley Region after filtering for valid geometries:")
    # print(delaware_valley_gdf['co_name'].unique())

    # Display the first few rows of the filtered GeoDataFrame
    # print("Counties in the Delaware Valley Region:")
    # print(delaware_valley_gdf.head())
else:
    print("Error: No shapefile (.shp) found in the folder.")


In [9]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import time

# Load the schools data
# Here, be sure to change the file path below to the respective Schools Database Excel file on your local machine.
schools = pd.read_excel('/Users/jazminb./Desktop/RA_Fall2024/AtlasData/SchoolsDatabase.xlsx')

# Initialize geolocator
geolocator = Nominatim(user_agent="school_locator")

# Function to get latitude and longitude using the automated geolocator with manual entry option.
def get_coordinates(name, address):
    # Check if address is missing
    # Automated geolocator uses the address to get the latitude and longitude
    if not address or pd.isna(address) or address.strip() == "":
        print(f"Address is missing for {name}.")
        # Prompt the user to input an address if missing
        address = input(f"Please input the address for {name}: ").strip()
        if not address:
            print(f"No address provided for {name}. Skipping.")
            return None, None, None  # Skip if no address is provided
    try:
        # Attempt to geocode the address
        location = geolocator.geocode(address)
        if location:
            print(f"Geocoded {name} successfully.")
            return address, location.latitude, location.longitude
        else:
            print(f"Could not geocode: {name} at {address}")
            # This is the manual entry option if the automated geolocator can not find the information.
            # IMPORTANT:  MUST have coordinates!!!
            # Latitude and longitude are REQUIRED to be able to plot the schools on the map. 
            manual_entry = input(f"Enter coordinates for {name} (format: 'latitude,longitude') or leave blank to skip: ").strip()
            if manual_entry:
                try:
                    lat, lon = map(float, manual_entry.split(','))
                    return address, lat, lon
                except ValueError:
                    print(f"Invalid format for {name}. Skipping.")
    except Exception as e:
        print(f"Error for {name} at {address}: {e}")
    return None, None, None

# Dataframe to store results
results = []

# Loop through schools DataFrame using .iterrows()
for _, school in schools.iterrows():
    name = school["name"]
    address = school["address"]
    school_type = school["type"]
    county = school["county"]
    
    # Get coordinates for each school
    updated_address, lat, lon = get_coordinates(name, address)
    results.append({
        "name": name,
        "type": school_type,
        "address": updated_address,
        "county": county,
        "latitude": lat,
        "longitude": lon
    })
    time.sleep(1)  # Pause to respect API usage limits

# Convert results to a DataFrame
school_coords = pd.DataFrame(results)

# We will save the updated schools data to the Excel file
# Here, be sure to change the file path below to the respective Schools Database Excel file on your local machine.
updated_file_path = '/Users/jazminb./Desktop/RA_Fall2024/AtlasData/SchoolsDatabase.xlsx'
school_coords.to_excel(updated_file_path, index=False)

# Define colors for each school type
type_colors = {
    "District Public": "blue",
    "Charter Public": "green",
    "Secular Private": "purple",
    "Religious Private": "red"
}

# Create a map centered on Philadelphia
m = folium.Map(location=[39.9526, -75.1652], zoom_start=12, width='75%', height='75%')

# Add a GeoJSON layer if needed (e.g., for a boundary or region)
# folium.GeoJson(delaware_valley_gdf).add_to(m)  # Uncomment if you have a GeoDataFrame

# Plot each school on the map with the correct color based on its type
for _, row in school_coords.iterrows():
    if not pd.isna(row["latitude"]) and not pd.isna(row["longitude"]):
        # Set the color based on the type of school
        marker_color = type_colors.get(row["type"], "gray")  # Default to gray if type is not found
        
        
        
        # Add marker to the map with the correct color
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=f"{row['name']} ({row['type']})<br>County: {row['county']}",
            tooltip=f"{row['name']} ({row['type']}) - {row['county']}",  # Shows name, type, and county in the tooltip
            icon=folium.Icon(color=marker_color)  # Apply the color
        ).add_to(m)

# Display the map

m

Address is missing for A.L. Fitzpatrick School.
No address provided for A.L. Fitzpatrick School. Skipping.
Address is missing for Abram S. Jenks School.
No address provided for Abram S. Jenks School. Skipping.
Address is missing for Add B. Anderson School.
No address provided for Add B. Anderson School. Skipping.
Address is missing for Alain Locke School.
No address provided for Alain Locke School. Skipping.
Address is missing for Albert M. Greenfield School.
No address provided for Albert M. Greenfield School. Skipping.
Address is missing for Alexander Adaire School.
No address provided for Alexander Adaire School. Skipping.
Address is missing for Alexander K. McClure School.
No address provided for Alexander K. McClure School. Skipping.
Address is missing for Allen M. Stearne School.
No address provided for Allen M. Stearne School. Skipping.
Address is missing for Andrew Hamilton School.
No address provided for Andrew Hamilton School. Skipping.
Address is missing for Andrew J. Morris

In [4]:
# ** Recommended: HTML Export for Cleaner Visibility **

# For a better and cleaner view of the notebook's output, you can
# export the `SACJResearch_SchoolAtlas.ipynb` to HTML. This will 
# remove code contents, leaving only the title, description, and 
# visualizations, making it more suitable for presentations or reports.

# Use the following command in the terminal command line when 
# finished updating to export the notebook to HTML:

# ```bash
# jupyter nbconvert --to html --TemplateExporter.exclude_input=True --no-input --TemplateExporter.exclude_output=False SACJResearch_SchoolAtlas.ipynb
# ```